In [25]:
import numpy as np
import pyspark as ps

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, explode
from pyspark.sql.functions import isnan, when, count, col


from pyspark.sql.types import IntegerType

In [2]:

spark = (ps.sql.SparkSession.builder 
        .master("local[4]") 
        .appName("wikiclimber") 
        .getOrCreate()
        )

In [8]:
df = spark.read.csv("/home/jovyan/work/Documents/dsi/8a_kaggle/dev_imac2/ascents_spark.csv", header=True)
# df = df.fillna(0)

In [27]:
df = df.withColumn("user_id", df["user_id"].cast(IntegerType()))


In [28]:
df = df.withColumn("rating", df["rating"].cast(IntegerType()))

In [29]:
df = df.withColumn("date", df["date"].cast(IntegerType()))

In [30]:
df.show()

+---+--------------------+-------+------+---------+
|_c0|                name|user_id|rating|     date|
+---+--------------------+-------+------+---------+
|  0|      The King And I|      1|     0|918342000|
|  1|              vet ej|      1|     0|925509600|
|  2|              Mr Big|      1|     0|933026400|
|  3|       Tak ska du ha|      1|     0|933026400|
|  4|       Korpen flyger|      1|     0|933458400|
|  5|           Valentine|      1|     0|914022000|
|  6|             Namnlös|      1|     0|915404400|
|  7|Nuat Hin (Massage...|      1|     0|916268400|
|  8|           Circus Oz|      1|     0|916441200|
|  9|       Primal scream|      1|     0|917478000|
| 10|     Fit to be Thaid|      1|     0|917650800|
| 11|            Nylander|      1|     0|925423200|
| 12|         Farmor Anka|      1|     0|925250400|
| 13|              Vet ej|      1|     0|925509600|
| 14|              Vet ej|      1|     0|925596000|
| 15|              Vet ej|      1|     0|925596000|
| 16|       

In [31]:
df.columns

['_c0', 'name', 'user_id', 'rating', 'date']

In [11]:
# df.drop("notes","raw_notes","total_score",'year','last_year', 'rec_date','project_ascent_date', 'crag_id','sector_id','comment','description','yellow_id','climb_try','repeat',
#        'exclude_from_ranking','user_recommended','chipped', 'shoe', 'video', 'screename').printSchema()

In [12]:
dir(df)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_collect_as_arrow',
 '_jcols',
 '_jdf',
 '_jmap',
 '_jseq',
 '_lazy_rdd',
 '_repr_html_',
 '_sc',
 '_schema',
 '_sort_cols',
 '_support_repr_html',
 '_to_corrected_pandas_type',
 'agg',
 'alias',
 'approxQuantile',
 'cache',
 'checkpoint',
 'coalesce',
 'colRegex',
 'collect',
 'columns',
 'corr',
 'count',
 'cov',
 'createGlobalTempView',
 'createOrReplaceGlobalTempView',
 'createOrReplaceTempView',
 'createTempView',
 'crossJoin',
 'crosstab',
 'cube',
 'describe',
 'distinct',
 'drop',
 'dropDuplicates',
 'drop_duplicates',
 'dropna',
 'dtypes',
 'exceptAll',
 'explain',
 'fillna',
 'fi

In [32]:
df.count()

4111879

In [14]:
# df = df.dropna()

In [15]:
df.count()

4111879

In [33]:
df.show()

+---+--------------------+-------+------+---------+
|_c0|                name|user_id|rating|     date|
+---+--------------------+-------+------+---------+
|  0|      The King And I|      1|     0|918342000|
|  1|              vet ej|      1|     0|925509600|
|  2|              Mr Big|      1|     0|933026400|
|  3|       Tak ska du ha|      1|     0|933026400|
|  4|       Korpen flyger|      1|     0|933458400|
|  5|           Valentine|      1|     0|914022000|
|  6|             Namnlös|      1|     0|915404400|
|  7|Nuat Hin (Massage...|      1|     0|916268400|
|  8|           Circus Oz|      1|     0|916441200|
|  9|       Primal scream|      1|     0|917478000|
| 10|     Fit to be Thaid|      1|     0|917650800|
| 11|            Nylander|      1|     0|925423200|
| 12|         Farmor Anka|      1|     0|925250400|
| 13|              Vet ej|      1|     0|925509600|
| 14|              Vet ej|      1|     0|925596000|
| 15|              Vet ej|      1|     0|925596000|
| 16|       

In [34]:
df

DataFrame[_c0: string, name: string, user_id: int, rating: int, date: int]

In [35]:
df.createOrReplaceTempView('ratings')
global_rating = spark.sql('''
    SELECT AVG(rating) as global_mean
    FROM ratings
    ''')

global_mean = global_rating.collect()[0].global_mean
global_mean

1.364091652613869

In [36]:

user_result = spark.sql('''
    SELECT user_id, avg(rating) as avg_user_rating
    FROM ratings
    group by user_id
    order by user_id

    ''')

user_result.show()

+-------+--------------------+
|user_id|     avg_user_rating|
+-------+--------------------+
|   null|   25127.74193548387|
|      1|  0.7428571428571429|
|      2|                 1.0|
|      3| 0.32786885245901637|
|      4|  0.6833333333333333|
|      5|  1.2802056555269923|
|      6|  1.1612903225806452|
|     10|  1.1655266757865936|
|     11|0.007194244604316547|
|     12| 0.28205128205128205|
|     13|                 0.0|
|     14|                 0.0|
|     15|  2.2901234567901234|
|     16|  1.8895348837209303|
|     17|                 0.6|
|     18|  1.6422764227642277|
|     19|  0.5396825396825397|
|     24|                 2.0|
|     25|                 2.0|
|     26|  1.6666666666666667|
+-------+--------------------+
only showing top 20 rows



In [19]:
# df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

In [37]:

n=10
user_result.collect()[n-1].avg_user_rating #gives avg user rating for nth user

0.28205128205128205

In [38]:

route_result = spark.sql('''
    SELECT name, AVG(rating) as avg_route_rating
    FROM ratings
    GROUP BY name
    ORDER BY name
    ''')

route_result.show()
# route_result.collect()[n-1].avg_movie_rating #gives avg movie rating for nth movie

+--------------------+------------------+
|                name|  avg_route_rating|
+--------------------+------------------+
|                null|0.8990536277602523|
|	 Massacre À La T...|               0.0|
|          	Dinosaure|               1.0|
|    	Earth to Stella|               1.0|
|    	Express sumarqu|               3.0|
|    	Indigo Galliace|               3.0|
|    	La farse tranqu|               2.0|
|    	Move Over Bruss|               0.0|
|	Prostowanie wariant|               1.0|
|         	Slip Bouse|               2.0|
|          	Spit Bull|               2.0|
|    	gagae de ma par|               3.0|
|    	peur aux tafiol|               3.0|
|    	priez pour nous|               3.0|
|              ....|               0.0|
| 	 Aresta dinamitada|               3.0|
| 	 Io sto con gli ip|               0.0|
| 	 Le P'tit Toit ...|               3.0|
| 	 Unendliche Ges...|               3.0|
|      	Aire de Repos|               3.0|
+--------------------+------------

In [39]:
route_result = spark.sql('''
    SELECT name, AVG(rating) as avg_route_rating
    FROM ratings
    GROUP BY name
    ORDER BY name
    ''')

# movie_result.show()
route_result = route_result.toPandas()

In [40]:
route_result

,name,avg_route_rating
0,None,0.899054
1,\t Massacre À La Tronconneuse,0.000000
2,\tDinosaure,1.000000
3,\tEarth to Stella,1.000000
4,\tExpress sumarqu,3.000000
...,...,...
886932,�zlem,1.500000
886933,���,0.000000
886934,🔥 in the Hole,2.000000
886935,😀,0.000000


# Modeling

In [41]:
training_df, test_df = df.randomSplit([0.8, 0.2])

In [42]:

als = ALS(rank=4 , maxIter=10 , regParam=0.1 , userCol='user_id', itemCol='name', \
          coldStartStrategy='drop', nonnegative = True, ratingCol='rating')
model = als.fit(training_df)

IllegalArgumentException: requirement failed: Column name must be of type numeric but was actually of type string.

In [ ]:
predictions = model.transform(test_df)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
new_predictions = predictions.filter(predictions.prediction!=np.nan)
rmse = evaluator.evaluate(predictions)
print(rmse)

In [ ]:
userRecs = model.recommendForAllUsers(10)
userRecs.show()